In [2]:
import pandas as pd
import numpy as np

In [15]:
#print("/Accident data/US_Accidents_Dec19.csv")
accidents_source=pd.read_csv("Accident data/US_Accidents_Dec19.csv")


TypeError: 'numpy.int32' object is not callable

In [16]:
accidents_source.size

145742415

In [17]:
accidents_source.shape

(2974335, 49)

In [18]:
accidents_source.describe

<bound method NDFrame.describe of                 ID    Source    TMC  Severity           Start_Time  \
0              A-1  MapQuest  201.0         3  2016-02-08 05:46:00   
1              A-2  MapQuest  201.0         2  2016-02-08 06:07:59   
2              A-3  MapQuest  201.0         2  2016-02-08 06:49:27   
3              A-4  MapQuest  201.0         3  2016-02-08 07:23:34   
4              A-5  MapQuest  201.0         2  2016-02-08 07:39:07   
...            ...       ...    ...       ...                  ...   
2974330  A-2974354      Bing    NaN         2  2019-08-23 18:03:25   
2974331  A-2974355      Bing    NaN         2  2019-08-23 19:11:30   
2974332  A-2974356      Bing    NaN         2  2019-08-23 19:00:21   
2974333  A-2974357      Bing    NaN         2  2019-08-23 19:00:21   
2974334  A-2974358      Bing    NaN         2  2019-08-23 18:52:06   

                    End_Time  Start_Lat   Start_Lng   End_Lat    End_Lng  ...  \
0        2016-02-08 11:00:00  39.865147  -84

In [20]:
accidents_source.columns


Index(['ID', 'Source', 'TMC', 'Severity', 'Start_Time', 'End_Time',
       'Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng', 'Distance(mi)',
       'Description', 'Number', 'Street', 'Side', 'City', 'County', 'State',
       'Zipcode', 'Country', 'Timezone', 'Airport_Code', 'Weather_Timestamp',
       'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)',
       'Visibility(mi)', 'Wind_Direction', 'Wind_Speed(mph)',
       'Precipitation(in)', 'Weather_Condition', 'Amenity', 'Bump', 'Crossing',
       'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station',
       'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop',
       'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight',
       'Astronomical_Twilight'],
      dtype='object')

In [23]:
target_columns=['Severity','Start_Time','End_Time','Start_Lat', 'Start_Lng','City', 'County', 'State',\
                'Zipcode', 'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)',\
       'Visibility(mi)', 'Wind_Direction', 'Wind_Speed(mph)','Precipitation(in)', 'Weather_Condition','Sunrise_Sunset']
accident_target=accidents_source[target_columns]

In [24]:
accident_target.head()

,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,City,County,State,Zipcode,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Sunrise_Sunset
0,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,Dayton,Montgomery,OH,45424,36.9,NaN,91.0,29.68,10.0,Calm,NaN,0.02,Light Rain,Night
1,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,Reynoldsburg,Franklin,OH,43068-3402,37.9,NaN,100.0,29.65,10.0,Calm,NaN,0.00,Light Rain,Night
2,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,Williamsburg,Clermont,OH,45176,36.0,33.3,100.0,29.67,10.0,SW,3.5,NaN,Overcast,Night
3,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,Dayton,Montgomery,OH,45417,35.1,31.0,96.0,29.64,9.0,SW,4.6,NaN,Mostly Cloudy,Night
4,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,Dayton,Montgomery,OH,45459,36.0,33.3,89.0,29.65,6.0,SW,3.5,NaN,Mostly Cloudy,Day


In [27]:
accident_target.to_csv("TargetData/us_accident_clean.csv")